importing file

In [1]:
import pandas as pd
import numpy as np

sheets = pd.read_excel('DE5_wells.xlsx', sheet_name=None, header=None)

for key in sheets:
    print(sheets[key])

       0      1      2      3      4      5      6      7      8      9
0  520 A  612 A  520 B  612 B  520 C  612 C  520 D  612 D  611 A  619 A
1  560 A  613 A  560 B  613 B  560 C  613 C  560 D  613 D  611 B  619 B
2  563 A  614 A  563 B  614 B  563 C  614 C  563 D  614 D  611 C  619 C
3  564 A  615 A  564 B  615 B  564 C  615 C  564 D  615 D  611 D  619 D
4    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  620 A
5  594 A  616 A  594 B  616 B  594 C  616 C  594 D  616 D    NaN  620 B
6  595 A  617 A  595 B  617 B  595 C  617 C  595 D  617 D    NaN  620 C
7  596 A  618 A  596 B  618 B  596 C  618 C  596 D  618 D    NaN  620 D
     0    1    2    3    4    5    6    7    8   9
0   P0   P0   P0   P0   P0   P0   P0   P0   P0  P0
1   P0   P0   P0   P0   P0   P0   P0   P0   P0  P0
2   P0   P0   P0   P0   P0   P0   P0   P0   P0  P0
3   P0   P0   P0   P0   P0   P0   P0   P0   P0  P0
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  P0
5   P0   P0   P0   P0   P0   P0   P0   P0  NaN

generating barcodeGroups.csv file. For P0, P5, P10, P17, used barcode plates 1-4 respectively, so will use bcAddDict to add the appropriate number to the forward and reverse barcode values for each of these, as these barcode plates simply increment the barcode number in each row/column, and the position of each sample in each plate was the same (e.g., 520 A was in well A1 for P0, P5, P10, and P17 minipreps)

Adding a function that will add a header line to a CSV file

In [2]:
def add_header(csv, header):
    """
    given a preexisting CSV file, will add a new line at the
    top that has all blank columns except for the 2nd value, which
    will be the `header` string
    """
    with open(csv,'r+') as file:
        content = file.read()
        file.seek(0,0)
        file.write(f',{header}\n{content}')



In [3]:

barcodeGroupRowsList = []
bcAddDict = {'P0':(1,1), 'P5':(13,1), 'P10':(1,13), 'P17':(13,13)}
gensList = []

for row in range(0,8):
    for col in range(0,10):
        if row == 4:
            if col != 9:
                continue
        if (col==8) and (row in [4,5,6,7]):
            continue
        nameBase = str(sheets['plasmid'].iloc[row,col]).replace(' ', '')
        for timepoint in ['P0','P5','P10','P17']:
            tp = sheets[timepoint].iloc[row,col]
            generations = int(tp[1:]) * 8
            name = f'{nameBase}-g{generations}'
            fwdbc, rvsbc = np.array([col,row]) + np.array(bcAddDict[timepoint])
            fwdbc = 'bc'+str(fwdbc)
            rvsbc = 'bc'+str(rvsbc)
            label, generations = name.split('-g')
            if generations not in gensList:
                gensList.append(generations)
            barcodeGroupRowsList.append([name, fwdbc, rvsbc, label, generations])

OGgroupsDF = pd.DataFrame(barcodeGroupRowsList, columns=['barcodeGroup', 'fwd', 'rvs', 'label', 'generations'])
OGgroupsDF.drop(['label', 'generations'], axis='columns').to_csv('barcodeGroups_firstBatch.csv',index=False)

add_header('barcodeGroups_firstBatch.csv','barcode_types')

OGgroupsDF

,barcodeGroup,fwd,rvs,label,generations
0,520A-g0,bc1,bc1,520A,0
1,520A-g32,bc13,bc1,520A,32
2,520A-g72,bc1,bc13,520A,72
3,520A-g120,bc13,bc13,520A,120
4,612A-g0,bc2,bc1,612A,0
...,...,...,...,...,...
267,618D-g128,bc20,bc20,618D,128
268,620D-g0,bc10,bc8,620D,0
269,620D-g24,bc22,bc8,620D,24
270,620D-g56,bc10,bc20,620D,56


now generating barcodeGroup csv file for the second round of p1 sequencing. For this I used the third barcoding plate, so fwd starts at 1 and rvs starts at 13

In [4]:
barcodeGroupRowsList = []
bcAdd = (1,13)
rows = 'ABCDEFGH'

for sampleRow in range(0,8):
    for sampleCol in range(0,7):
        tp, well = str(sheets['redo'].iloc[sampleRow,sampleCol]).split(' ')
        if tp == 'water':
            continue
        OGrow = rows.find(well[0])
        OGcol = int(well[1:])-1
        OGtimepoint = str(sheets[tp].iloc[OGrow,OGcol])
        generations = int(OGtimepoint[1:]) * 8

        nameBase = str(sheets['plasmid'].iloc[OGrow,OGcol]).replace(' ', '')
        name = f'{nameBase}-g{generations}'
        fwdbc, rvsbc = np.array([sampleCol,sampleRow]) + np.array(bcAdd)
        fwdbc = 'bc'+str(fwdbc)
        rvsbc = 'bc'+str(rvsbc)
        label, generations = name.split('-g')
        barcodeGroupRowsList.append([name, fwdbc, rvsbc, label, generations])

redoGroupsDF = pd.DataFrame(barcodeGroupRowsList, columns=['barcodeGroup', 'fwd', 'rvs', 'label', 'generations'])
redoGroupsDF.drop(['label', 'generations'], axis='columns').to_csv('barcodeGroups_redo.csv',index=False)
add_header('barcodeGroups_redo.csv','barcode_types')

making barcodeGroup csv file for the TPDNAP sequencing. Used barcode plate 1 for P5 and barcode plate 2 for P17

In [5]:
barcodeGroupRowsList = []
bcAddDict = {'P5':(1,1), 'P17':(13,1)}

for row in range(0,8):
    for col in range(0,10):
        if row == 4:
            if col != 9:
                continue
        if (col==8) and (row in [4,5,6,7]):
            continue
        nameBase = str(sheets['plasmid'].iloc[row,col]).replace(' ', '')
        for timepoint in ['P5','P17']:
            tp = sheets[timepoint].iloc[row,col]
            generations = int(tp[1:]) * 8
            name = f'{nameBase}-g{generations}'
            fwdbc, rvsbc = np.array([col,row]) + np.array(bcAddDict[timepoint])
            fwdbc = 'bc'+str(fwdbc)
            rvsbc = 'bc'+str(rvsbc)
            label, generations = name.split('-g')
            barcodeGroupRowsList.append([name, fwdbc, rvsbc, label, generations])

TPDNAPgroupsDF = pd.DataFrame(barcodeGroupRowsList, columns=['barcodeGroup', 'fwd', 'rvs', 'label', 'generations'])
TPDNAPgroupsDF.drop(['label', 'generations'], axis='columns').to_csv('barcodeGroups_TPDNAP.csv',index=False)
add_header('barcodeGroups_TPDNAP.csv','barcode_types')

Now generating timepoints .csv files for analysis of each of the different regions of p1. Using samples from the redo sequencing if they exist

In [6]:
gensList = [int(x) for x in gensList]
gensList.sort()
# plasmidDict = {'520':'KS', '560':'TKS', '563':'SFSGTKIS', '564':'TRKSY', '594':'SG+RTRKSY', '595':'SGI+RTRKSY', '596':'SFSGI+RTRKSY', '611':'DTRKSY', '612':'RTRTSY', '613':'RTRSSY', '614':'DKRTSRKSY', '615':'T
# plasmidDict = {'520':'0-520-611', '560':'1-560-611+P680T', '563':'2-563-SgtKis', '564':'2-564-rTrixy', '594':'2.5-594-SG+RTRKSY', '595':'2.5-595-SGI+RTRKSY', '596':'2.5-596-SFSGI+RTRKSY', '611':'3-611-DRTRIXY', '612':'3-612-RTRTSY', '613':'3-613-RTRSSY', '614':'3-614-TrixyDks', '615':'3-615-TrixyILV', '616':'4-616-SFSGI+DRTRIXY', '617':'4-617-SFSGI+RTRTSY', '618':'4-618-SFSGI+DRTRIXY', '619':'4-619-SFSGI+TrixyDks', '620':'4-620-SFSGI+TrixyILV'}
plasmidDict = {'520':'I777K-L900S', '560':'TKS', '563':'SgtKis', '564':'Trixy', '594':'BadBoy1', '595':'BadBoy2', '596':'BadBoy3', '611':'3A', '612':'3B', '613':'3C', '614':'3D', '615':'3E', '616':'BB-3A', '617':'BB-3B', '618':'BB-Tv', '619':'BB-5k', '620':'BB-3E'}

for gene in ['', '-LEU2', '-URA3', '-mScarlettI']:
    timepointsRowsList = []
    for label in OGgroupsDF['label'].unique():
        labeltpList = [int(label[:-1]), plasmidDict[label[:-1]]]
        labeldf = OGgroupsDF[OGgroupsDF['label']==label]
        # generate rows of timepoint csv file, using samples from the redo sequencing if they exist, or the original sequencing if not
        for tp in gensList:
            group = f'{label}-g{tp}'
            if group in list(redoGroupsDF['barcodeGroup']):
                labeltpList.append(f'DE5redo{gene}_{group}')
            elif group in list(OGgroupsDF['barcodeGroup']):
                labeltpList.append(f'DE5{gene}_{group}')
            else:
                labeltpList.append(np.nan)
        timepointsRowsList.append(labeltpList)

    tpdf = pd.DataFrame(timepointsRowsList, columns=['plasmidID', 'barcodeGroup']+gensList)
    tpdf = tpdf.sort_values('plasmidID', ascending=True).drop(columns='plasmidID')
    tpdf.to_csv(f'timepoints{gene}.csv', index=False)
    add_header(f'timepoints{gene}.csv','generation')

tpdf

,barcodeGroup,0,24,32,40,56,64,72,80,104,112,120,128
0,I777K-L900S,DE5-mScarlettI_520A-g0,NaN,DE5redo-mScarlettI_520A-g32,NaN,NaN,NaN,DE5redo-mScarlettI_520A-g72,NaN,NaN,NaN,DE5redo-mScarlettI_520A-g120,NaN
2,I777K-L900S,DE5-mScarlettI_520B-g0,NaN,DE5-mScarlettI_520B-g32,NaN,NaN,NaN,DE5-mScarlettI_520B-g72,NaN,NaN,NaN,DE5redo-mScarlettI_520B-g120,NaN
4,I777K-L900S,DE5redo-mScarlettI_520C-g0,NaN,DE5-mScarlettI_520C-g32,NaN,NaN,NaN,DE5-mScarlettI_520C-g72,NaN,NaN,NaN,DE5redo-mScarlettI_520C-g120,NaN
6,I777K-L900S,DE5-mScarlettI_520D-g0,NaN,DE5redo-mScarlettI_520D-g32,NaN,NaN,NaN,DE5-mScarlettI_520D-g72,NaN,NaN,NaN,DE5redo-mScarlettI_520D-g120,NaN
16,TKS,DE5-mScarlettI_560D-g0,NaN,NaN,DE5-mScarlettI_560D-g40,NaN,NaN,NaN,DE5-mScarlettI_560D-g80,NaN,NaN,NaN,DE5-mScarlettI_560D-g128
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,10k-3D,DE5-mScarlettI_619A-g0,NaN,DE5-mScarlettI_619A-g32,NaN,NaN,DE5-mScarlettI_619A-g64,NaN,NaN,NaN,DE5redo-mScarlettI_619A-g112,NaN,NaN
49,10k-3E,DE5-mScarlettI_620B-g0,DE5-mScarlettI_620B-g24,NaN,NaN,DE5-mScarlettI_620B-g56,NaN,NaN,NaN,DE5redo-mScarlettI_620B-g104,NaN,NaN,NaN
40,10k-3E,DE5-mScarlettI_620A-g0,DE5-mScarlettI_620A-g24,NaN,NaN,DE5-mScarlettI_620A-g56,NaN,NaN,NaN,DE5redo-mScarlettI_620A-g104,NaN,NaN,NaN
58,10k-3E,DE5-mScarlettI_620C-g0,DE5-mScarlettI_620C-g24,NaN,NaN,DE5redo-mScarlettI_620C-g56,NaN,NaN,NaN,DE5redo-mScarlettI_620C-g104,NaN,NaN,NaN
